# Web scraper for China Government Statistical Report

- reports comes from [China Statistical Information](http://www.tjcn.org/tjgbsy/nd/36944.html)
- will all Chinese cities' report of 2020, some cities are missing, 2019 report will be used.

In [1]:
import pandas as pd
import numpy as np
from time import sleep

import requests
from bs4 import BeautifulSoup  
# url for 2021 Statistics report China Cities
url_2019 = "http://www.tjcn.org/tjgbsy/nd/36163.html"
url_2020 = "http://www.tjcn.org/tjgbsy/nd/36541.html"

response = requests.get(url_2020)  
soup = BeautifulSoup(response.content, 'html.parser')  

response = requests.get(url_2019)  
soup_old = BeautifulSoup(response.content, 'html.parser')  

### scrap index page to get all cities' link

In [2]:
table = soup.find_all('td', class_='xwnr')[0] 
html_cities = [row.find_all('td')[1] for row in table.find_all('tr')]
cities = []
for row in html_cities:
    cities += row.find_all('a')
    
df = pd.DataFrame()
for city in cities:
    df.loc[city.text, 'link'] = "http://www.tjcn.org" + city.get('href')  

table = soup_old.find_all('td', class_='xwnr')[0] 
html_cities = [row.find_all('td')[1] for row in table.find_all('tr')]
cities = []
for row in html_cities:
    cities += row.find_all('a')
    
df_old = pd.DataFrame()
for city in cities:
    df_old.loc[city.text, 'link'] = "http://www.tjcn.org" + city.get('href')  

df = df.merge(df_old, left_index=True, right_index=True, suffixes = ('_2020', '_2019'))
df['link'] = df.apply(lambda x: x['link_2019'] if x['link_2020'] is None else x['link_2020'], axis=1)

### crawling each city's page information

In [3]:
for city in df.index:
    response = requests.get(df.loc[city, 'link'])
    soup = BeautifulSoup(response.content, 'html.parser')
    df.loc[city, 'text'] = soup.text
    sleep(1)
    print(f"<{city}>", end="")

<北京市><天津市><上海市><重庆市><石家庄市><唐山市><秦皇岛市><邯郸市><邢台市><保定市><张家口市><承德市><沧州市><廊坊市><衡水市><太原市><大同市><阳泉市><长治市><晋城市><朔州市><晋中市><运城市><忻州市><临汾市><吕梁市><呼和浩特市><包头市><乌海市><赤峰市><通辽市><鄂尔多斯市><呼伦贝尔市><巴彦淖尔市><乌兰察布市><兴安盟><锡林郭勒盟><阿拉善盟><沈阳市><大连市><鞍山市><抚顺市><本溪市><丹东市><锦州市><营口市><阜新市><辽阳市><盘锦市><铁岭市><朝阳市><葫芦岛市><长春市><吉林市><四平市><辽源市><通化市><白山市><松原市><白城市><延边州><哈尔滨市><齐齐哈尔市><鸡西市><鹤岗市><双鸭山市><大庆市><伊春市><佳木斯市><七台河市><牡丹江市><黑河市><绥化市><大兴安岭地区><南京市><无锡市><徐州市><常州市><苏州市><南通市><连云港市><淮安市><盐城市><扬州市><镇江市><泰州市><宿迁市><杭州市><宁波市><温州市><嘉兴市><湖州市><绍兴市><金华市><衢州市><舟山市><台州市><丽水市><合肥市><芜湖市><蚌埠市><淮南市><马鞍山市><淮北市><铜陵市><安庆市><黄山市><滁州市><阜阳市><宿州市><六安市><亳州市><池州市><宣城市><福州市><厦门市><莆田市><三明市><泉州市><漳州市><南平市><龙岩市><宁德市><南昌市><景德镇市><萍乡市><九江市><新余市><鹰潭市><赣州市><吉安市><宜春市><抚州市><上饶市><济南市><青岛市><淄博市><枣庄市><东营市><烟台市><潍坊市><济宁市><泰安市><威海市><日照市><临沂市><德州市><聊城市><滨州市><菏泽市><郑州市><开封市><洛阳市><平顶山市><安阳市><鹤壁市><新乡市><濮阳市><许昌市><漯河市><三门峡市><南阳市><商丘市><信阳市><周口市><驻马店市><武汉市><黄石市><十堰市><宜昌市><襄阳市><荆州市><荆门市><鄂州市><孝感市><黄冈市><咸宁市><随州市><恩施州><长沙市><株洲市><湘潭市><衡阳市><邵阳市><岳阳市><常德市><张家界市><益阳市><郴州市><永州市><怀化市><娄底市><湘西州><

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<防城港市><钦州市><贵港市><玉林市><百色市><贺州市><河池市><来宾市><崇左市><海口市><三亚市><儋州市><成都市><自贡市><攀枝花市><泸州市><德阳市><绵阳市><广元市><遂宁市><内江市><乐山市><南充市><眉山市><宜宾市><广安市><达州市><雅安市><巴中市><资阳市><阿坝州><甘孜州><凉山州><贵阳市><六盘水市><遵义市><安顺市><毕节市><铜仁市><黔西南州><黔东南州><黔南州><昆明市><曲靖市><玉溪市><保山市><昭通市><普洱市><临沧市><楚雄州><文山州><西双版纳州><大理州><德宏州><怒江州><迪庆州><拉萨市><日喀则市><林芝市><山南市><西安市><铜川市><宝鸡市><咸阳市><渭南市><延安市><汉中市><榆林市><安康市><商洛市><兰州市><嘉峪关市><金昌市><白银市><天水市><武威市><张掖市><平凉市><酒泉市><庆阳市><定西市><陇南市><临夏州><甘南州><西宁市><海东市><海北州><黄南州><海南州><果洛州><玉树州><海西州><银川市><石嘴山市><吴忠市><固原市><中卫市><乌鲁木齐市><克拉玛依市><吐鲁番市><哈密市><昌吉州><博尔塔拉州><阿克苏地区><克孜勒苏州><喀什地区><和田地区><伊犁州><塔城地区><阿勒泰地区><新疆生产建设兵团>

In [4]:
df.to_csv('city.csv')